In [2]:
import numpy as np
import requests as rq
import pandas as pd

In [3]:

from nba_api.stats.endpoints import playercareerstats
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import leagueleaders
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import leaguestandings
from nba_api.stats.static import teams


In [61]:
## player id --> create a loop for this to grap the current year stats
## season averages

career_stat_columns = ['FG3M', 'REB', 'AST', 'STL', 'BLK', 'PTS']
careers = pd.DataFrame()
for i in range(len(home_rosters)):
# for i in range(2):

    career = playercareerstats.PlayerCareerStats(home_rosters['PLAYER_ID'][i]).get_data_frames()[0]
    # print(home_rosters['PLAYER_ID'][i])
    # Divide each selected column by the number of games played
    per_game_averages = career[career_stat_columns].div(career['GP'], axis=0)
    # print(per_game_averages)
    per_game_averages.insert(0, 'PLAYER_ID', career['PLAYER_ID'])
    per_game_averages.insert(0, 'SEASON_ID', career['SEASON_ID'])
    per_game_averages.insert(2, 'TEAM_ID', career['TEAM_ID'])
    per_game_averages.insert(3, 'TEAM_ABBREVIATION', career['TEAM_ABBREVIATION'])

    careers = pd.concat([careers, per_game_averages.tail(1)], axis=0, ignore_index=True)

careers.to_csv('home_players_careers.csv', index=False)

## Save but needs a loop


In [58]:
careers

,SEASON_ID,PLAYER_ID,TEAM_ID,TEAM_ABBREVIATION,FG3M,REB,AST,STL,BLK,PTS
0,2023-24,1630169,1610612754,IND,3.162791,3.906977,11.674419,1.023256,0.697674,21.837209
1,2023-24,203081,1610612749,MIL,2.901961,4.137255,6.725490,1.039216,0.196078,24.627451
2,2023-24,1628369,1610612738,BOS,3.057692,8.557692,4.788462,1.000000,0.596154,27.134615
3,2023-24,1630178,1610612755,PHI,3.040816,3.653061,6.428571,0.959184,0.510204,25.693878
4,2023-24,1630567,1610612761,TOR,1.781818,8.145455,5.963636,1.218182,1.454545,20.054545
5,2023-24,1631094,1610612753,ORL,1.545455,6.854545,5.254545,1.054545,0.527273,23.000000
6,2023-24,1627759,1610612738,BOS,1.940000,5.460000,3.740000,1.080000,0.560000,21.980000
7,2023-24,1628973,1610612752,NYK,2.673077,3.826923,6.461538,0.923077,0.192308,27.576923
8,2023-24,1629027,1610612737,ATL,3.340000,2.740000,10.920000,1.400000,0.220000,26.660000
9,2023-24,1628389,1610612748,MIA,0.022222,10.644444,4.177778,1.066667,0.955556,20.222222


In [25]:
home_rosters['PLAYER_ID'][0]

1630169

In [6]:
## Games of the Day

games = scoreboard.ScoreBoard().get_dict()['scoreboard']['games']
# games_today.get_json()
# games_today.get_dict()['scoreboard']['games'][0]
columns = ['HomeTeamID', 'HomeTeam', 'HomeTeamWins', 'HomeTeamLosses', 'AwayTeamID', 'AwayTeam', 'AwayTeamWins', 'AwayTeamLosses']
games_today = pd.DataFrame(index=range(len(games)), columns = columns)

for i in range(len(games)):
    games_today['HomeTeamID'][i] = games[i]['homeTeam']['teamId']
    games_today['AwayTeamID'][i] = games[i]['awayTeam']['teamId']
    games_today['HomeTeam'][i] = games[i]['homeTeam']['teamTricode']
    games_today['AwayTeam'][i] = games[i]['awayTeam']['teamTricode']
    games_today['HomeTeamWins'][i] = games[i]['homeTeam']['wins']
    games_today['AwayTeamWins'][i] = games[i]['awayTeam']['wins']
    games_today['HomeTeamLosses'][i] = games[i]['homeTeam']['losses']
    games_today['AwayTeamLosses'][i] = games[i]['awayTeam']['losses']

games_today

## Save run 1st

,HomeTeamID,HomeTeam,HomeTeamWins,HomeTeamLosses,AwayTeamID,AwayTeam,AwayTeamWins,AwayTeamLosses
0,1610616833,EST,1,0,1610616834,WST,0,1


In [7]:
## data frame of all player IDs that are playing at home and away -->


home_teams = games_today['HomeTeamID'].to_list()
away_teams = games_today['AwayTeamID'].to_list()

home_opp=[]
away_opp=[]

columns = ['TeamID', 'PLAYER', 'POSITION', 'PLAYER_ID']
home_rosters = pd.DataFrame(columns = columns)
away_rosters = pd.DataFrame(columns = columns)

for team in range(len(home_teams)):
    home_rosters = pd.concat([home_rosters, commonteamroster.CommonTeamRoster(team_id=f"{home_teams[team]}").get_data_frames()[0][columns]], ignore_index=True)
    

for team in range(len(away_teams)):
    away_rosters = pd.concat([away_rosters, commonteamroster.CommonTeamRoster(team_id=f"{away_teams[team]}").get_data_frames()[0][columns]], ignore_index=True)
    

for player in range(len(home_rosters)):
    for i in range(len(home_teams)):
        if games_today['HomeTeamID'][i] == home_rosters['TeamID'][player]:
            home_opp.append(games_today['AwayTeam'][i])
          
for player in range(len(away_rosters)):
    for i in range(len(away_teams)):
        if games_today['AwayTeamID'][i] == away_rosters['TeamID'][player]:
            away_opp.append(games_today['HomeTeam'][i])          
            
home_opp = pd.DataFrame(home_opp, columns=['HomeOpp'])
home_rosters = pd.concat([home_rosters, home_opp], axis=1)

away_opp = pd.DataFrame(away_opp, columns=['AwayOpp'])
away_rosters = pd.concat([away_rosters, away_opp], axis=1)

## Save this will get home/away rosters

In [23]:
home_rosters


,TeamID,PLAYER,POSITION,PLAYER_ID,HomeOpp
0,1610616833,Tyrese Haliburton,G,1630169,WST
1,1610616833,Damian Lillard,G,203081,WST
2,1610616833,Jayson Tatum,F-G,1628369,WST
3,1610616833,Tyrese Maxey,G,1630178,WST
4,1610616833,Scottie Barnes,F-G,1630567,WST
5,1610616833,Paolo Banchero,F,1631094,WST
6,1610616833,Jaylen Brown,G-F,1627759,WST
7,1610616833,Jalen Brunson,G,1628973,WST
8,1610616833,Trae Young,G,1629027,WST
9,1610616833,Bam Adebayo,C-F,1628389,WST


In [67]:
## home stats

home_stats_df = pd.DataFrame()
GP =[]

stat_columns = ['FG3M', 'REB', 'AST', 'STL', 'BLK', 'PTS']

# for player in range(12,14):
for player in range(len(home_rosters)):
    # print(player)
    ## game_log will get the player throughout the season
    game_log = playergamelog.PlayerGameLog(player_id=home_rosters['PLAYER_ID'][player]).get_data_frames()[0]
   

       ## home and away statistics
    home_games = game_log[~game_log['MATCHUP'].str.contains('@')]

    try:
        home_stats = home_games[selected_columns].sum()/len(home_games)
    except:
        home_stats = pd.Series([0, 0, 0, 0, 0, 0], index= selected_columns)
    
    home_stats_df = pd.concat([home_stats_df, home_stats.to_frame().transpose()], ignore_index=True )
    GP.append(len(home_games))
    
home_stats_df.insert(0, 'GP', GP)
    
    
home_player_stats = pd.concat([home_rosters,home_stats_df], axis=1)
home_player_stats.to_csv('home_player_stats.csv', index=False)
## Save pulls the stats for each home player

In [68]:
home_player_stats

,TeamID,PLAYER,POSITION,PLAYER_ID,HomeOpp,GP,FG3M,REB,AST,STL,BLK,PTS
0,1610616833,Tyrese Haliburton,G,1630169,WST,23,2.695652,3.521739,11.086957,1.086957,0.695652,20.782609
1,1610616833,Damian Lillard,G,203081,WST,28,3.107143,4.357143,6.250000,1.071429,0.178571,25.321429
2,1610616833,Jayson Tatum,F-G,1628369,WST,28,3.214286,8.071429,4.928571,0.821429,0.642857,27.000000
3,1610616833,Tyrese Maxey,G,1630178,WST,27,3.148148,3.444444,6.481481,0.888889,0.518519,25.370370
4,1610616833,Scottie Barnes,F-G,1630567,WST,27,1.555556,9.222222,6.037037,1.185185,1.555556,21.148148
5,1610616833,Paolo Banchero,F,1631094,WST,26,1.576923,6.846154,5.384615,0.884615,0.461538,21.423077
6,1610616833,Jaylen Brown,G-F,1627759,WST,26,2.000000,5.576923,4.000000,1.076923,0.576923,22.038462
7,1610616833,Jalen Brunson,G,1628973,WST,25,2.440000,3.680000,6.920000,1.000000,0.200000,27.640000
8,1610616833,Trae Young,G,1629027,WST,28,3.035714,2.357143,10.785714,1.535714,0.285714,25.500000
9,1610616833,Bam Adebayo,C-F,1628389,WST,23,0.043478,10.086957,4.565217,0.869565,1.260870,19.478261


In [19]:
away_stats_df = pd.DataFrame()
GP =[]
stat_columns = ['FG3M', 'REB', 'AST', 'STL', 'BLK', 'PTS']
# for player in range(12,14):
for player in range(len(away_rosters)):
    # print(player)
    ## game_log will get the player throughout the season
    game_log = playergamelog.PlayerGameLog(player_id=away_rosters['PLAYER_ID'][player]).get_data_frames()[0]
  
       ## home and away statistics
    away_games = game_log[game_log['MATCHUP'].str.contains('@')]

    try:
        away_stats = away_games[selected_columns].sum()/len(away_games)
    except:
        home_stats = pd.Series([0, 0, 0, 0, 0, 0], index= selected_columns)
        away_stats = pd.Series([0, 0, 0, 0, 0, 0], index= selected_columns)
    
    away_stats_df = pd.concat([away_stats_df, away_stats.to_frame().transpose()], ignore_index=True )
    GP.append(len(away_games))
away_stats_df.insert(0, 'GP', GP)
    
    
away_player_stats = pd.concat([away_rosters,away_stats_df], axis=1)

#Save pulls the stats for each away player


In [16]:
away_player_stats

,TeamID,PLAYER,POSITION,PLAYER_ID,AwayOpp,GP,FG3M,REB,AST,STL,BLK,PTS
0,1610616834,Devin Booker,G,1626164,EST,22,2.272727,4.772727,6.409091,0.909091,0.545455,29.636364
1,1610616834,Shai Gilgeous-Alexander,G,1628983,EST,27,1.333333,5.148148,6.074074,1.888889,0.925926,30.666667
2,1610616834,Kawhi Leonard,F,202695,EST,25,2.680000,6.440000,3.240000,1.640000,0.880000,26.040000
3,1610616834,Anthony Davis,F-C,203076,EST,24,0.500000,12.416667,3.791667,1.083333,2.500000,24.875000
4,1610616834,Anthony Edwards,G,1630162,EST,29,2.689655,5.344828,5.379310,1.137931,0.413793,26.379310
5,1610616834,Paul George,F,202331,EST,25,3.400000,4.720000,3.360000,1.800000,0.480000,22.440000
6,1610616834,Nikola Jokic,C,203999,EST,27,1.296296,12.074074,8.370370,1.074074,0.925926,26.666667
7,1610616834,LeBron James,F,2544,EST,22,2.045455,7.636364,6.954545,1.272727,0.863636,26.500000
8,1610616834,Stephen Curry,G,201939,EST,24,4.916667,4.500000,5.541667,0.916667,0.375000,26.791667
9,1610616834,Karl-Anthony Towns,C-F,1626157,EST,30,2.200000,8.400000,3.133333,0.600000,0.666667,23.266667


In [13]:
# last ten games played
last_ten = game_log.head(10)


## the MIL will have to be the input of whatever team they will play today
## this will be for a specific player as well
matchups = game_log[game_log['MATCHUP'].str.contains(home_rosters['HomeOpp'][player])]



In [17]:
matchups


,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE


In [20]:
## All teams and their standings
league_standings = leaguestandings.LeagueStandings().get_data_frames()[0]
league_standings

,LeagueID,SeasonID,TeamID,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,...,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,PreAS,PostAS
0,00,22023,1610612738,Boston,Celtics,East,28-6,1,,Atlantic,...,None,None,None,None,None,3-0,11-4,12-2,41-12,None
1,00,22023,1610612750,Minnesota,Timberwolves,West,25-8,1,,Northwest,...,None,None,None,None,None,1-2,13-2,10-3,37-16,None
2,00,22023,1610612739,Cleveland,Cavaliers,East,23-11,2,,Central,...,None,None,None,None,None,1-3,9-6,8-5,35-17,None
3,00,22023,1610612760,Oklahoma City,Thunder,West,22-13,2,,Northwest,...,None,None,None,None,None,3-1,9-5,10-3,36-17,None
4,00,22023,1610612746,LA,Clippers,West,21-13,3,,Pacific,...,None,None,None,None,None,3-1,5-9,11-2,35-17,None
5,00,22023,1610612749,Milwaukee,Bucks,East,25-12,3,,Central,...,None,None,None,None,None,2-1,11-5,11-2,35-19,None
6,00,22023,1610612743,Denver,Nuggets,West,20-12,4,,Northwest,...,None,None,None,None,None,4-0,9-6,10-5,36-18,None
7,00,22023,1610612752,New York,Knicks,East,22-11,4,,Atlantic,...,None,None,None,None,None,2-2,9-5,6-8,33-21,None
8,00,22023,1610612740,New Orleans,Pelicans,West,23-18,5,,Southwest,...,None,None,None,None,None,2-1,8-8,9-5,32-22,None
9,00,22023,1610612755,Philadelphia,76ers,East,21-13,5,,Atlantic,...,None,None,None,None,None,2-1,10-5,10-4,32-21,None


In [61]:
## players on teams
## input team id --> then get players positions and IDs

team_players = commonteamroster.CommonTeamRoster(team_id='1610612751') 
team = team_players.get_data_frames()[0]

bkn_roster = team[['TeamID', 'PLAYER', 'POSITION', 'PLAYER_ID']]
## this will be a series for each
## to use probably should turn it into a list

bkn_roster

,TeamID,PLAYER,POSITION,PLAYER_ID
0,1610612751,Dariq Whitehead,F,1641727
1,1610612751,Mikal Bridges,G-F,1628969
2,1610612751,Cameron Johnson,F,1629661
3,1610612751,Dennis Smith Jr.,G,1628372
4,1610612751,Lonnie Walker IV,G-F,1629022
5,1610612751,Trendon Watford,F,1630570
6,1610612751,Ben Simmons,G-F,1627732
7,1610612751,Keita Bates-Diop,F,1628966
8,1610612751,Dennis Schroder,G,203471
9,1610612751,Day'Ron Sharpe,C,1630549


In [96]:
## player id --> game by game stats
## player id --> matchup using the opposing 'MIL'

## inputs needed player id --> check
## matchup of the day



game_log = playergamelog.PlayerGameLog(player_id='203999').get_data_frames()[0]
career = playercareerstats.PlayerCareerStats(player_id='203999') 
selected_columns = ['FG3M', 'REB', 'AST', 'STL', 'BLK', 'PTS']
seasons=career.get_data_frames()[0]

## last ten games played
last_ten = game_log.head(10)


## the MIL will have to be the input of whatever team they will play today
matchups = game_log[game_log['MATCHUP'].str.contains('MIL')]


## home and away statistics
home_games = game_log[~game_log['MATCHUP'].str.contains('@')]
away_games = game_log[game_log['MATCHUP'].str.contains('@')]

home_stats = home_games[selected_columns].sum()/len(home_games)
away_stats = away_games[selected_columns].sum()/len(home_games)



# Season Averages
per_game_averages = seasons[selected_columns].div(seasons['GP'], axis=0)

per_game_averages.insert(0, 'PLAYER_ID', seasons['PLAYER_ID'])
per_game_averages.insert(0, 'SEASON_ID', seasons['SEASON_ID'])
per_game_averages.insert(2, 'TEAM_ID', seasons['TEAM_ID'])
per_game_averages.insert(3, 'TEAM_ABBREVIATION', seasons['TEAM_ABBREVIATION'])


,SEASON_ID,PLAYER_ID,TEAM_ID,TEAM_ABBREVIATION,FG3M,REB,AST,STL,BLK,PTS
0,2015-16,203999,1610612743,DEN,0.350000,7.000000,2.362500,0.987500,0.625000,9.950000
1,2016-17,203999,1610612743,DEN,0.616438,9.835616,4.917808,0.835616,0.753425,16.726027
2,2017-18,203999,1610612743,DEN,1.480000,10.706667,6.106667,1.200000,0.813333,18.466667
3,2018-19,203999,1610612743,DEN,1.037500,10.812500,7.250000,1.350000,0.687500,20.050000
4,2019-20,203999,1610612743,DEN,1.095890,9.739726,7.013699,1.164384,0.602740,19.945205
5,2020-21,203999,1610612743,DEN,1.277778,10.833333,8.319444,1.319444,0.666667,26.361111
6,2021-22,203999,1610612743,DEN,1.310811,13.770270,7.891892,1.472973,0.851351,27.081081
7,2022-23,203999,1610612743,DEN,0.826087,11.840580,9.826087,1.260870,0.681159,24.492754
8,2023-24,203999,1610612743,DEN,1.056604,12.018868,8.943396,1.150943,0.924528,26.075472


In [80]:
## all nba teams

# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = teams.get_teams()
all_teams = pd.DataFrame(nba_teams)
all_teams

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [96]:
from nba_api.stats.endpoints import teamdashboardbygeneralsplits
n = pd.DataFrame()
# for team in range(len(all_teams)):
for team in range(0,3):
    print(team)
    new = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(all_teams['id'].to_list()[team]).get_data_frames()[0]
    print(new)
    n = pd.concat([n, new], axis=0)
    print(n)
n

0
  GROUP_SET GROUP_VALUE SEASON_YEAR  GP   W   L  W_PCT     MIN   FGM   FGA  \
0   Overall     2023-24     2023-24  55  24  31  0.436  2660.0  2397  5144   

   ...  REB_RANK  AST_RANK  TOV_RANK  STL_RANK  BLK_RANK  BLKA_RANK  PF_RANK  \
0  ...         1         1         1         1         1          1        1   

   PFD_RANK  PTS_RANK  PLUS_MINUS_RANK  
0         1         1                1  

[1 rows x 55 columns]
  GROUP_SET GROUP_VALUE SEASON_YEAR  GP   W   L  W_PCT     MIN   FGM   FGA  \
0   Overall     2023-24     2023-24  55  24  31  0.436  2660.0  2397  5144   

   ...  REB_RANK  AST_RANK  TOV_RANK  STL_RANK  BLK_RANK  BLKA_RANK  PF_RANK  \
0  ...         1         1         1         1         1          1        1   

   PFD_RANK  PTS_RANK  PLUS_MINUS_RANK  
0         1         1                1  

[1 rows x 55 columns]
1
  GROUP_SET GROUP_VALUE SEASON_YEAR  GP   W   L  W_PCT     MIN   FGM   FGA  \
0   Overall     2023-24     2023-24  55  43  12  0.782  2665.0  2391  49

,GROUP_SET,GROUP_VALUE,SEASON_YEAR,GP,W,L,W_PCT,MIN,FGM,FGA,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,Overall,2023-24,2023-24,55,24,31,0.436,2660.0,2397,5144,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,55,43,12,0.782,2665.0,2391,4967,...,1,1,1,1,1,1,1,1,1,1
0,Overall,2023-24,2023-24,53,36,17,0.679,2554.0,2267,4698,...,1,1,1,1,1,1,1,1,1,1


In [104]:
from nba_api.stats.endpoints import leaguedashteamstats
offense_team_stats = leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0]
new.columns

Index(['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS',
       'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
       'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
       'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK'],
      dtype='object')

In [105]:
offense_team_stats['PFD_RANK']

0      5
1     26
2     29
3     30
4     18
5     23
6      2
7     20
8     24
9     19
10    10
11    16
12    17
13     4
14    13
15    12
16     6
17     7
18    14
19     9
20    15
21     1
22    11
23     3
24    27
25    21
26    22
27    25
28     8
29    28
Name: PFD_RANK, dtype: int64

In [107]:
from nba_api.stats.endpoints import playerdashboardbyteamperformance
n = playerdashboardbyteamperformance.PlayerDashboardByTeamPerformance().get_data_frames()

TypeError: PlayerDashboardByTeamPerformance.__init__() missing 1 required positional argument: 'player_id'

In [108]:
from nba_api.stats.endpoints import leaguedashptteamdefend
n = leaguedashptteamdefend.LeagueDashPtTeamDefend().get_data_frames()

In [110]:
from nba_api.stats.endpoints import teamgamestreakfinder
n = teamgamestreakfinder.TeamGameStreakFinder().get_data_frames()

In [111]:
n


[                            TEAM_NAME     TEAM_ID  GAMESTREAK  \
 0                   San Antonio Spurs  1610612759        3363   
 1                  Los Angeles Lakers  1610612747        3361   
 2                  Philadelphia 76ers  1610612755        3360   
 3                    Dallas Mavericks  1610612742        3359   
 4               Golden State Warriors  1610612744        3351   
 ..                                ...         ...         ...   
 95                 Capital City Go-Go  1612709928         175   
 96  New Orleans/Oklahoma City Hornets  1610612740         164   
 97                  Kansas City Kings  1610612758         164   
 98                     Colorado 14ers  1612709901         150   
 99                    Anaheim Arsenal  1612709899         150   
 
               STARTDATE              ENDDATE  ACTIVESTREAK  NUMSEASONS  \
 0   1983-10-29T00:00:00  2024-02-14T00:00:00             1          41   
 1   1983-10-28T00:00:00  2024-02-14T00:00:00           